In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import linprog
from joblib import Parallel, delayed
import os
import tempfile
from RiskMeasureEst import *
import random
from Env import Env
from Agent import *
from tqdm import tqdm

In [2]:
gamma = 0.9
aph = 0.50
B = 6
p1 = 0.25
O = 1

In [3]:
p = np.zeros((B,3,B))

from scipy.stats import binom
import numpy as np

def binomial_pmf_all(n, p):
    k_values = np.arange(n + 1)
    return binom.pmf(k_values, n, p)

for sidx in range(B):
    for aidx in range(3):
            p[sidx,aidx] = binomial_pmf_all(B-1,p1)

In [4]:
r = np.zeros((B,3,B))

for sidx in range(B):
    for aidx in range(3):
        for s1idx in range(B):
            if sidx < s1idx:
                r[sidx,aidx,s1idx] = aidx-1
            elif s1idx < sidx:
                r[sidx,aidx,s1idx] = -(aidx-1)
            else:
                r[sidx,aidx,s1idx] = -abs(aidx-1)

In [5]:
Priors = []

alpha0 =  np.zeros((B,3,B))

for sidx in range(B):
    for aidx in range(3):
        alpha0[sidx,aidx] = binomial_pmf_all(B-1,0.5)

Priors.append(alpha0)

alpha0 =  np.zeros((B,3,B))

for sidx in range(B):
    for aidx in range(3):
        alpha0[sidx,aidx] = binomial_pmf_all(B-1,0.4)

Priors.append(alpha0)

In [6]:
def train(p,r,B,A,alpha0,
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='1',
                  task='CT'):
    
    action_array = np.zeros((B,1000,20))
    
    for i in range(20):
        print('step,',i)
        env = Env(p,-r,B=B,A=A)
        agent = Agent(env,alpha0.copy(),N=N,
                      aph=aph, \
                      kappa = kappa,\
                      risk_type=risk_type,\
                      aph_beta = aph_beta,\
                      beta_type=beta_type)

        for j in tqdm(range(1000)):
            if j%100 == 0:
                agent.DP()

            s0idx,aidx,s1idx, cost = agent.env.step(agent.selectAction(env.sidx))
            agent.updateAlpha(s0idx,aidx,s1idx)
            
            action_array[:,j,i] = agent.Q.argmin(axis=1)
            
    np.save('res/rho_{}_beta_{}_Prior_{}_task_{}'.format(risk_type,beta_type,Prior_name,task),\
            action_array)

In [7]:
train(p,r,6,3,Priors[0],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='1',
                  task = 'CT')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 98.02it/s]


step, 1


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 126.66it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 80.56it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 54.47it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.39it/s]


In [8]:
train(p,r,6,3,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='2',
                  task = 'CT')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 85.56it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 72.25it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 70.50it/s]


step, 18


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 50.41it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 98.58it/s]


In [9]:
train(p,r,6,3,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='Mean',
                  aph_beta = 0.2,
                  beta_type='Mean',
                  Prior_name='2',
                  task = 'CT')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 75.20it/s]


step, 1


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 103.41it/s]


step, 2


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 94.08it/s]


step, 18


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 108.26it/s]


step, 19


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 109.26it/s]


In [10]:
train(p,r,6,3,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='Mean',
                  aph_beta = 0.2,
                  beta_type='CVaR',
                  Prior_name='2',
                  task = 'CT')

step, 0


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 84.54it/s]


step, 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 88.46it/s]


step, 2


...

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 136.49it/s]


step, 18


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 104.88it/s]


step, 19


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 86.43it/s]


In [11]:
train(p,r,6,3,Priors[1],
                  N=10,
                  aph=0.5, 
                  kappa = None,
                  risk_type='CVaR',
                  aph_beta = 0.2,
                  beta_type='CVaR',
                  Prior_name='2',
                  task = 'CT')

step, 0


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 200.20it/s]


step, 1


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 196.00it/s]


step, 2


...

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 173.28it/s]


step, 18


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 185.93it/s]


step, 19


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 156.29it/s]
